# Safest Route Assement Model Training

**Working Model**

### First test

In [5]:
import pandas as pd
import numpy as np
import folium
import networkx as nx
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

# Define the towns and their coordinates
towns = {
    'Nairobi': (-1.286389, 36.817223),
    'Mombasa': (-4.043477, 39.668206),
    'Nakuru': (-0.303099, 36.066285),
    'Eldoret': (0.514277, 35.269065),
    'Kisumu': (-0.091702, 34.767956),
    'Nyeri': (-0.421250, 36.945752),
    'Machakos': (-1.516820, 37.266485),
    'Kericho': (-0.366690, 35.291340),
    'Meru': (-0.047200, 37.648000),
    'Kitale': (1.002559, 34.986032),
    'Garissa': (-0.456550, 39.664640),
    'Isiolo': (0.353073, 37.582666),
    'Bungoma': (0.591278, 34.564658),
    'Wajir': (1.737327, 40.065940),
    'Mandera': (3.930530, 41.855910),
    'Malindi': (-3.219186, 40.116944),
    'Lamu': (-2.271189, 40.902012),
    'Thika': (-1.033349, 37.069328),
    'Namanga': (-2.545290, 36.792530),
    'Kitui': (-1.374818, 38.010555),
    'Naivasha': (-0.707222, 36.431944),
    'Narok': (-1.078850, 35.860000),
    'Busia': (0.4605, 34.1115),
    'Bomet': (-0.7827, 35.3428),
    'Marsabit': (2.3342, 37.9891),
    'Voi': (-3.3962, 38.5565)
}

# Define multiple routes with at least 3 paths leading to each town
routes = [
    ('Nairobi', 'Mombasa', 'A109'),
    ('Nairobi', 'Nakuru', 'A104'),
    ('Nairobi', 'Eldoret', 'A104'),
    ('Nairobi', 'Nyeri', 'B5'),
    ('Nairobi', 'Garissa', 'A3'),
    ('Mombasa', 'Malindi', 'A14'),
    ('Mombasa', 'Garissa', 'B8'),
    ('Nakuru', 'Eldoret', 'A104'),
    ('Nakuru', 'Kericho', 'B1'),
    ('Nakuru', 'Nyeri', 'B5'),
    ('Eldoret', 'Kitale', 'B2'),
    ('Eldoret', 'Kisumu', 'A104'),
    ('Kisumu', 'Bungoma', 'A1'),
    ('Kisumu', 'Busia', 'A1'),
    ('Kisumu', 'Kericho', 'B1'),
    ('Nyeri', 'Nairobi', 'B5'),
    ('Nyeri', 'Meru', 'B6'),
    ('Garissa', 'Wajir', 'A3'),
    ('Garissa', 'Mandera', 'A3'),
    ('Kitale', 'Bungoma', 'A1'),
    ('Machakos', 'Nairobi', 'B6'),
    ('Machakos', 'Mombasa', 'A109'),
    ('Machakos', 'Kitui', 'B7'),
    ('Meru', 'Isiolo', 'A2'),
    ('Meru', 'Nairobi', 'B6'),
    ('Kericho', 'Nakuru', 'B1'),
    ('Kericho', 'Kisumu', 'B1'),
    ('Isiolo', 'Marsabit', 'A2'),
    ('Isiolo', 'Garissa', 'A2'),
    ('Bungoma', 'Kitale', 'A1'),
    ('Bungoma', 'Busia', 'A1'),
    ('Wajir', 'Garissa', 'A3'),
    ('Wajir', 'Mandera', 'A3'),
    ('Mandera', 'Wajir', 'A3'),
    ('Thika', 'Nairobi', 'A2'),
    ('Thika', 'Nyeri', 'A2'),
    ('Naivasha', 'Nairobi', 'B3'),
    ('Naivasha', 'Narok', 'C88'),
    ('Lamu', 'Malindi', 'C112')
]

# Generate simulated crime data
def generate_crime_data():
    data = []
    for origin, destination, road_name in routes:
        origin_lat, origin_lon = towns[origin]
        dest_lat, dest_lon = towns[destination]

        for day in range(730):  # 2 years of data
            day_of_week = day % 7  # Day of the week (0=Monday, 6=Sunday)
            hour_of_day = random.randint(0, 23)  # Random hour of the day
            crime_occurred = np.random.choice([0, 1], p=[0.85, 0.15])  # 15% chance of crime

            data.append({
                'Origin': origin,
                'Destination': destination,
                'Road_Name': road_name,
                'Day_of_Week': day_of_week,
                'Hour_of_Day': hour_of_day,
                'Crime_Occurred': crime_occurred,
                'Origin_Lat': origin_lat,
                'Origin_Lon': origin_lon,
                'Destination_Lat': dest_lat,
                'Destination_Lon': dest_lon
            })
    return pd.DataFrame(data)

crime_data = generate_crime_data()

# Train a crime prediction model
X = crime_data[['Day_of_Week', 'Hour_of_Day']]
y = crime_data['Crime_Occurred']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Function to find the safest route
def find_safest_route(origin, destination, day_of_week, hour_of_day):
    G = nx.Graph()

    for _, row in crime_data.iterrows():
        G.add_edge(row['Origin'], row['Destination'], weight=row['Crime_Occurred'])

    safest_path = nx.shortest_path(G, source=origin, target=destination, weight='weight')

    print(f"Safest Route from {origin} to {destination}: {safest_path}")

# Example usage:
find_safest_route('Nairobi', 'Kisumu', day_of_week=2, hour_of_day=14)


Model Accuracy: 0.85
Safest Route from Nairobi to Kisumu: ['Nairobi', 'Eldoret', 'Kisumu']


In [ ]:
find_safest_route('Nairobi', 'Kisumu', day_of_week=2, hour_of_day=14)

### Increase number of towns

In [6]:
import pandas as pd
import numpy as np
import folium
import networkx as nx
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
import warnings
warnings.filterwarnings("ignore")

# Define the towns and their coordinates
towns = {
    'Nairobi': (-1.286389, 36.817223),
    'Mombasa': (-4.043477, 39.668206),
    'Nakuru': (-0.303099, 36.066285),
    'Eldoret': (0.514277, 35.269065),
    'Kisumu': (-0.091702, 34.767956),
    'Nyeri': (-0.421250, 36.945752),
    'Machakos': (-1.516820, 37.266485),
    'Kericho': (-0.366690, 35.291340),
    'Meru': (-0.047200, 37.648000),
    'Kitale': (1.002559, 34.986032),
    'Garissa': (-0.456550, 39.664640),
    'Isiolo': (0.353073, 37.582666),
    'Bungoma': (0.591278, 34.564658),
    'Wajir': (1.737327, 40.065940),
    'Mandera': (3.930530, 41.855910),
    'Malindi': (-3.219186, 40.116944),
    'Lamu': (-2.271189, 40.902012),
    'Thika': (-1.033349, 37.069328),
    'Namanga': (-2.545290, 36.792530),
    'Kitui': (-1.374818, 38.010555),
    'Naivasha': (-0.707222, 36.431944),
    'Narok': (-1.078850, 35.860000),
    'Busia': (0.4605, 34.1115),
    'Bomet': (-0.7827, 35.3428),
    'Marsabit': (2.3342, 37.9891),
    'Voi': (-3.3962, 38.5565)
}

# Define multiple routes with at least 3 paths leading to each town
routes = [
    ('Nairobi', 'Mombasa', 'A109'),
    ('Nairobi', 'Nakuru', 'A104'),
    ('Nairobi', 'Eldoret', 'A104'),
    ('Nairobi', 'Nyeri', 'B5'),
    ('Nairobi', 'Garissa', 'A3'),
    ('Mombasa', 'Malindi', 'A14'),
    ('Mombasa', 'Garissa', 'B8'),
    ('Nakuru', 'Eldoret', 'A104'),
    ('Nakuru', 'Kericho', 'B1'),
    ('Nakuru', 'Nyeri', 'B5'),
    ('Eldoret', 'Kitale', 'B2'),
    ('Eldoret', 'Kisumu', 'A104'),
    ('Kisumu', 'Bungoma', 'A1'),
    ('Kisumu', 'Busia', 'A1'),
    ('Kisumu', 'Kericho', 'B1'),
    ('Nyeri', 'Nairobi', 'B5'),
    ('Nyeri', 'Meru', 'B6'),
    ('Garissa', 'Wajir', 'A3'),
    ('Garissa', 'Mandera', 'A3'),
    ('Kitale', 'Bungoma', 'A1'),
    ('Machakos', 'Nairobi', 'B6'),
    ('Machakos', 'Mombasa', 'A109'),
    ('Machakos', 'Kitui', 'B7'),
    ('Meru', 'Isiolo', 'A2'),
    ('Meru', 'Nairobi', 'B6'),
    ('Kericho', 'Nakuru', 'B1'),
    ('Kericho', 'Kisumu', 'B1'),
    ('Isiolo', 'Marsabit', 'A2'),
    ('Isiolo', 'Garissa', 'A2'),
    ('Bungoma', 'Kitale', 'A1'),
    ('Bungoma', 'Busia', 'A1'),
    ('Wajir', 'Garissa', 'A3'),
    ('Wajir', 'Mandera', 'A3'),
    ('Mandera', 'Wajir', 'A3'),
    ('Thika', 'Nairobi', 'A2'),
    ('Thika', 'Nyeri', 'A2'),
    ('Naivasha', 'Nairobi', 'B3'),
    ('Naivasha', 'Narok', 'C88'),
    ('Lamu', 'Malindi', 'C112')
]

# Generate simulated crime data
def generate_crime_data():
    data = []
    for origin, destination, road_name in routes:
        origin_lat, origin_lon = towns[origin]
        dest_lat, dest_lon = towns[destination]

        for day in range(730):  # 2 years of data
            day_of_week = day % 7  # Day of the week (0=Monday, 6=Sunday)
            hour_of_day = random.randint(0, 23)  # Random hour of the day
            crime_occurred = np.random.choice([0, 1], p=[0.85, 0.15])  # 15% chance of crime

            data.append({
                'Origin': origin,
                'Destination': destination,
                'Road_Name': road_name,
                'Day_of_Week': day_of_week,
                'Hour_of_Day': hour_of_day,
                'Crime_Occurred': crime_occurred,
                'Origin_Lat': origin_lat,
                'Origin_Lon': origin_lon,
                'Destination_Lat': dest_lat,
                'Destination_Lon': dest_lon
            })
    return pd.DataFrame(data)

crime_data = generate_crime_data()

# Train a crime prediction model
X = crime_data[['Day_of_Week', 'Hour_of_Day']]
y = crime_data['Crime_Occurred']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Function to predict crime risk for a specific road
def predict_crime_probability(road, day_of_week, hour_of_day):
    X_new = np.array([[day_of_week, hour_of_day]])
    predicted_risk = clf.predict_proba(X_new)[0][1]  # Probability of crime occurring
    return predicted_risk

# Find the safest route using predicted crime probabilities
def find_safest_route(origin, destination, day_of_week, hour_of_day):
    G = nx.Graph()

    # Add edges with predicted crime probabilities as weights
    for route in routes:
        o, d, road_name = route
        crime_prob = predict_crime_probability(road_name, day_of_week, hour_of_day)
        G.add_edge(o, d, weight=crime_prob, road=road_name)

    # Find the safest path (path with the lowest crime risk)
    safest_path = nx.shortest_path(G, source=origin, target=destination, weight='weight')

    # Build route description
    route_description = []
    for i in range(len(safest_path) - 1):
        edge_data = G.get_edge_data(safest_path[i], safest_path[i+1])
        route_description.append(f"Take {edge_data['road']} from {safest_path[i]} to {safest_path[i+1]}")

    # Output the safest route and its description
    print(f"Safest Route from {origin} to {destination}:")
    for step in route_description:
        print(step)

# Example usage: Safest route from Nairobi to Mombasa on a Tuesday at 2 PM
find_safest_route('Nairobi', 'Mombasa', day_of_week=2, hour_of_day=14)


Model Accuracy: 0.86
Safest Route from Nairobi to Mombasa:
Take A109 from Nairobi to Mombasa


In [7]:
# Example usage: Safest route from Nairobi to Mombasa on a Tuesday at 2 PM
find_safest_route('Nairobi', 'Busia', day_of_week=2, hour_of_day=14)

Safest Route from Nairobi to Busia:
Take A104 from Nairobi to Eldoret
Take A104 from Eldoret to Kisumu
Take A1 from Kisumu to Busia


In [8]:
import folium

# Function to display the safest route on a map
def display_route_on_map(route):
    # Initialize the map, centered on Kenya
    map_center = [0.0236, 37.9062]  # Center of Kenya
    kenya_map = folium.Map(location=map_center, zoom_start=6)

    # Add markers and lines for the safest route
    for i in range(len(route) - 1):
        origin = route[i]
        destination = route[i + 1]

        # Get the coordinates of origin and destination
        origin_coords = towns[origin]
        destination_coords = towns[destination]

        # Add markers for the towns
        folium.Marker(location=origin_coords, popup=origin, icon=folium.Icon(color='blue')).add_to(kenya_map)
        folium.Marker(location=destination_coords, popup=destination, icon=folium.Icon(color='blue')).add_to(kenya_map)

        # Draw lines between the towns
        folium.PolyLine(locations=[origin_coords, destination_coords], color='green', weight=5).add_to(kenya_map)

    return kenya_map

# Modified function to find the safest route and display it on the map
def find_safest_route_and_display(origin, destination, day_of_week, hour_of_day):
    G = nx.Graph()

    # Add edges with predicted crime probabilities as weights
    for route in routes:
        o, d, road_name = route
        crime_prob = predict_crime_probability(road_name, day_of_week, hour_of_day)
        G.add_edge(o, d, weight=crime_prob, road=road_name)

    # Find the safest path (path with the lowest crime risk)
    safest_path = nx.shortest_path(G, source=origin, target=destination, weight='weight')

    # Build route description
    route_description = []
    for i in range(len(safest_path) - 1):
        edge_data = G.get_edge_data(safest_path[i], safest_path[i+1])
        route_description.append(f"Take {edge_data['road']} from {safest_path[i]} to {safest_path[i+1]}")

    # Print the safest route and its description
    print(f"Safest Route from {origin} to {destination}:")
    for step in route_description:
        print(step)

    # Display the route on the map
    route_map = display_route_on_map(safest_path)
    return route_map

# Example usage: Safest route from Nairobi to Mombasa on a Tuesday at 2 PM
route_map = find_safest_route_and_display('Nairobi', 'Nyeri', day_of_week=2, hour_of_day=14)
route_map


Safest Route from Nairobi to Nyeri:
Take B5 from Nairobi to Nyeri


In [9]:
# Example usage: Safest route from Nairobi to Mombasa on a Tuesday at 2 PM
route_map = find_safest_route_and_display('Nairobi', 'Kisumu', day_of_week=2, hour_of_day=23)
route_map

Safest Route from Nairobi to Kisumu:
Take A104 from Nairobi to Eldoret
Take A104 from Eldoret to Kisumu


In [10]:

# Define towns and routes
towns = {
    'Nairobi': (1.286389, 36.817223),
    'Mombasa': (-4.043477, 39.668206),
    'Kisumu': (-0.091702, 34.767956),
    'Eldoret': (0.515310, 35.269385),
    'Nakuru': (-0.303099, 36.065945),
    'Kericho': (-0.368236, 35.295677),
    'Kakamega': (0.340988, 34.754274),
    'Meru': (0.050017, 37.641069),
    'Embu': (-0.541609, 37.456048),
    'Kitale': (1.003835, 34.987351),
    'Malindi': (-3.163533, 40.116198),
    'Garissa': (-0.455636, 39.664804),
    'Machakos': (-1.516313, 37.259322),
    'Isiolo': (0.350498, 37.580693),
    'Moyale': (3.504122, 39.083116)
}

routes = [
    ('Nairobi', 'Mombasa', 'A109'),
    ('Nairobi', 'Kisumu', 'A104/A1'),
    ('Nairobi', 'Eldoret', 'A104'),
    ('Nairobi', 'Nakuru', 'A104'),
    ('Nairobi', 'Kericho', 'B1'),
    ('Nairobi', 'Kakamega', 'A104'),
    ('Nairobi', 'Meru', 'B6'),
    ('Nairobi', 'Embu', 'B7'),
    ('Nairobi', 'Kitale', 'A104'),
    ('Nairobi', 'Malindi', 'A109'),
    ('Nairobi', 'Garissa', 'A3'),
    ('Nairobi', 'Machakos', 'B6'),
    ('Nairobi', 'Isiolo', 'A2'),
    ('Nairobi', 'Moyale', 'A2'),
    ('Mombasa', 'Kisumu', 'A109'),
    ('Mombasa', 'Eldoret', 'A109'),
    ('Mombasa', 'Nakuru', 'A109'),
    ('Mombasa', 'Kericho', 'B8'),
    ('Mombasa', 'Kakamega', 'B8'),
    ('Mombasa', 'Meru', 'B8'),
    ('Mombasa', 'Embu', 'B8'),
    ('Mombasa', 'Kitale', 'B8'),
    ('Mombasa', 'Malindi', 'B8'),
    ('Mombasa', 'Garissa', 'B8'),
    ('Mombasa', 'Machakos', 'B8'),
    ('Mombasa', 'Isiolo', 'B8'),
    ('Mombasa', 'Moyale', 'B8'),
    ('Kisumu', 'Eldoret', 'A1'),
    ('Kisumu', 'Nakuru', 'A1'),
    ('Kisumu', 'Kericho', 'A1'),
    ('Kisumu', 'Kakamega', 'A1'),
    ('Kisumu', 'Meru', 'A1'),
    ('Kisumu', 'Embu', 'A1'),
    ('Kisumu', 'Kitale', 'A1'),
    ('Kisumu', 'Malindi', 'A1'),
    ('Kisumu', 'Garissa', 'A1'),
    ('Kisumu', 'Machakos', 'A1'),
    ('Kisumu', 'Isiolo', 'A1'),
    ('Kisumu', 'Moyale', 'A1'),
    ('Eldoret', 'Nakuru', 'A104'),
    ('Eldoret', 'Kericho', 'B1'),
    ('Eldoret', 'Kakamega', 'B1'),
    ('Eldoret', 'Meru', 'B1'),
    ('Eldoret', 'Embu', 'B1'),
    ('Eldoret', 'Kitale', 'B1'),
    ('Eldoret', 'Malindi', 'B1'),
    ('Eldoret', 'Garissa', 'B1'),
    ('Eldoret', 'Machakos', 'B1'),
    ('Eldoret', 'Isiolo', 'B1'),
    ('Eldoret', 'Moyale', 'B1'),
    ('Nakuru', 'Kericho', 'B1'),
    ('Nakuru', 'Kakamega', 'B1'),
    ('Nakuru', 'Meru', 'B1'),
    ('Nakuru', 'Embu', 'B1'),
    ('Nakuru', 'Kitale', 'B1'),
    ('Nakuru', 'Malindi', 'B1'),
    ('Nakuru', 'Garissa', 'B1'),
    ('Nakuru', 'Machakos', 'B1'),
    ('Nakuru', 'Isiolo', 'B1'),
    ('Nakuru', 'Moyale', 'B1'),
    ('Kericho', 'Kakamega', 'B1'),
    ('Kericho', 'Meru', 'B1'),
    ('Kericho', 'Embu', 'B1'),
    ('Kericho', 'Kitale', 'B1'),
    ('Kericho', 'Malindi', 'B1'),
    ('Kericho', 'Garissa', 'B1'),
    ('Kericho', 'Machakos', 'B1'),
    ('Kericho', 'Isiolo', 'B1'),
    ('Kericho', 'Moyale', 'B1'),
    ('Kakamega', 'Meru', 'B1'),
    ('Kakamega', 'Embu', 'B1'),
    ('Kakamega', 'Kitale', 'B1'),
    ('Kakamega', 'Malindi', 'B1'),
    ('Kakamega', 'Garissa', 'B1'),
    ('Kakamega', 'Machakos', 'B1'),
    ('Kakamega', 'Isiolo', 'B1'),
    ('Kakamega', 'Moyale', 'B1'),
    ('Meru', 'Embu', 'B1'),
    ('Meru', 'Kitale', 'B1'),
    ('Meru', 'Malindi', 'B1'),
    ('Meru', 'Garissa', 'B1'),
    ('Meru', 'Machakos', 'B1'),
    ('Meru', 'Isiolo', 'B1'),
    ('Meru', 'Moyale', 'B1'),
    ('Embu', 'Kitale', 'B1'),
    ('Embu', 'Malindi', 'B1'),
    ('Embu', 'Garissa', 'B1'),
    ('Embu', 'Machakos', 'B1'),
    ('Embu', 'Isiolo', 'B1'),
    ('Embu', 'Moyale', 'B1'),
    ('Kitale', 'Malindi', 'B1'),
    ('Kitale', 'Garissa', 'B1'),
    ('Kitale', 'Machakos', 'B1'),
    ('Kitale', 'Isiolo', 'B1'),
    ('Kitale', 'Moyale', 'B1'),
    ('Malindi', 'Garissa', 'B1'),
    ('Malindi', 'Machakos', 'B1'),
    ('Malindi', 'Isiolo', 'B1'),
    ('Malindi', 'Moyale', 'B1'),
    ('Garissa', 'Machakos', 'B1'),
    ('Garissa', 'Isiolo', 'B1'),
    ('Garissa', 'Moyale', 'B1'),
    ('Machakos', 'Isiolo', 'B1'),
    ('Machakos', 'Moyale', 'B1'),
    ('Isiolo', 'Moyale', 'B1')
]

# Generate historical crime data
def generate_crime_data():
    data = []
    for origin, destination, road_name in routes:
        origin_lat, origin_lon = towns[origin]
        dest_lat, dest_lon = towns[destination]
        for day in range(7):  # Simulate each day of the week
            for hour in range(24):  # Simulate each hour of the day
                crime_occurred = np.random.choice([0, 1], p=[0.8, 0.2])
                data.append({
                    'Origin': origin,
                    'Destination': destination,
                    'Road_Name': road_name,
                    'Day_of_Week': day,
                    'Hour_of_Day': hour,
                    'Crime_Occurred': crime_occurred
                })
    return pd.DataFrame(data)

crime_data = generate_crime_data()

# Prepare features and labels
X = crime_data[['Day_of_Week', 'Hour_of_Day']]
y = crime_data['Crime_Occurred']

# Train a RandomForest model
model = RandomForestClassifier(n_estimators=100)
model.fit(X, y)

# Save the model
# joblib.dump(model, 'crime_prediction_model.pkl')


RandomForestClassifier()